## Рассчет стоимости 


Будем использовать готовые и открытые API:
1. OCR - yandex cloud -> 0.12 рублей за фоток
2. LLM - OpenRouter -> цена зависит от модели

In [1]:
text_receipt = """
ООО "ПРОДТОРГ"

690033, г. Владивосток, пр-кт 100-летия Владивостока, д. 68

КАССОВЫЙ ЧЕК 142

(ПРИХОД)

145.77

=145.77

9

9.99

*1

ХАРБИН СВ

139.90

139.90

1

ПАКЕТ РЕМИ сити 380

[M+10491 ВОДА ДЕТСКАЯ Н ДЕТО 19

[M+18076 ПИВО ХАРБИН СВ

[M+17732 ПИВО

M+12661 KBAC ЖИВОЙ АРС 149.98

M+16126

16126 KBAC ЖИВОГО БР 148.79

19797 НАПИТОК ЭНЕРГЕ 119.90

=139.90

=139.90

=149.98

=148.79

=119.90

*1.0

1

1

=41.76

=86.98

=55.90

=56.90

23.63

55.90

56.90

69.90 0.338

259.90 *0.142

229.90 *0.258

77.98

1

001095шт

153591шт

017921UT

017921шт

003779шт

148431шт

198822шт

045006шт

121403шт

181207шт

181206шт

032784кг

066284кг

066061кг

194296шт

081834шт

167506шт

004031кг

САЛФЕТКИ ВЛАЖНЫЕ ONE T 86.98

САЛФЕТКИ БУМАЖНЫЕ ДИВН 41.76

ЛЕПЕШКА С ЧЕСНОКОМ 350

ПЕПЕШКА УЗБЕКСКАЯ 350Г

ОГУРЦЫ ГРУНТОВЫЕ ПРИМОР

ПЕРЕЦ СЛАДКИЙ ОРАНЖЕВЫЙ

ПОМИДОРЫ ЧЕРРИ РОЗОВЫЕ

МОРОЖЕНОЕ МЕЛОНА ПУРПУ

[M+18969 МОРОЖЕНОЕ МELO 90.99

МОРОЖЕНОЕ БИНГРЭ БОДРЯ 94.98

ПЕРЕЦ СЛАДКИЙ КРАСНЫЙ К 189.90 0.346

=36.91

=59.31

=77.98

=90.99

=94.98

=65.71

БАНКОВСКИЕ ОПЛАТЫ

000 Продторг

г. Владивосток, Приморский край

пр-кт 100-летия Владивостока

д. 68

11:53

ЧЕК

0105

Оплата

Мерчант: 701000111801

(E4) *5977

A0000006581010

1545.28

06.07.25

ПАО СБЕРБАНК

Терминал: 32662488

MIR

Сумма (Руб):

Комиссия за операцию 0 руб.

ОДОБРЕНО

K/A: 059518

518755305974

Проверено на устройстве клиента

B51807A736F7DFCEB9299FC8C9070488B367819B

Итог

БЕЗНАЛИЧНЫМИ

СУММА НДС 20%

=1545.28

=1545.28

=155.38 СУММА НДС 10%

=55.73

=118.24

ВЫ СЭКОНОМИЛИ, руб

Спасибо за покупку!

Количество наклеек 2 шт.

Ваша карта лояльности 2555000676659

Активный баланс 185.0 бонусов

Вам начислено 46.4 бонусов

У Вас списано 0.0 бонусов

КАССИР Хайдарова

КАССИИ Хандарова 1og.gov.ru

САЙТ ФНС www.nalog govеми Сити"

МЕСТО РАСЧЕТОВ Ма1085 06.07.25 11:53 KACCA 0002.01 СМЕНА 1085 06

CHO OCH

ОД 128408 оп 0860959236

3H KKT 0128226843 он 728 2543111571
"""

In [29]:
tiktoken.get_encoding("cl100k_base")

<Encoding 'cl100k_base'>

In [34]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)



In [75]:
ocr_ya = 0.12   # Стоимость OCR для одного чека
count_receipts = 1 # Количество чеков для обработки
dollar_to_ruble = 80  # Примерный курс доллара к рублю

In [76]:
prompt_text = f"""
# Role
Ты — ассистент по извлечению данных из текстов чеков о покупках. Тебе дан текст чека, который может содержать список покупок, их стоимость и другую информацию (даты, адреса, скидки, итоговые суммы и т.д.).

# Goal
Твоя задача — выделить из чека только следующие данные для каждой купленной позиции:
-позиция — наименование товара или услуги
=цена позиции — итоговая цена этого товара или услуги (без учета скидок, если указано отдельно)

#Instruction
1. Игнорируй итоги, суммы по скидкам, налоги, способы оплаты и прочую информацию, не являющуюся отдельной покупкой.
2. Если у позиции указано количество и цена за штуку, указывай итоговую цену (цена × количество).
3. Не включай товары с нулевой ценой или подарки.
4. Сохраняй структуру: одна строка — одна позиция.

# Формат ответа:
Для каждого чека составь список в формате:
Название позиции - цена итоговая

# Чеки
# Чек 1
{text_receipt*count_receipts}
"""

In [ ]:
output_text_receipt = """ХАРБИН СВ - 139.90 - K x Цена за позицию
ПАКЕТ РЕМИ СИТИ 380 - 9.99
ВОДА ДЕТСКАЯ Н ДЕТО 19 - 10491
ПИВО ХАРБИН СВ - 18076
ПИВО - 17732
КВАС ЖИВОЙ АРС - 149.98
КВАС ЖИВОЙ БР - 148.79
НАПИТОК ЭНЕРГЕ - 119.90
САЛФЕТКИ ВЛАЖНЫЕ ONE T - 86.98
САЛФЕТКИ БУМАЖНЫЕ ДИВН - 41.76
ЛЕПЕШКА С ЧЕСНОКОМ 350 - 55.90
ЛЕПЕШКА УЗБЕКСКАЯ 350Г - 56.90
ОГУРЦЫ ГРУНТОВЫЕ ПРИМОР - 69.90
ПЕРЕЦ СЛАДКИЙ ОРАНЖЕВЫЙ - 259.90
ПОМИДОРЫ ЧЕРРИ РОЗОВЫЕ - 229.90
МОРОЖЕНОЕ МЕЛОНА ПУРПУ - 77.98
МОРОЖЕНОЕ MELO - 90.99
МОРОЖЕНОЕ БИНГРЭ БОДРЯ - 94.98
ПЕРЕЦ СЛАДКИЙ КРАСНЫЙ К - 65.71""" * count_receipts

In [ ]:
prompt_input_tokens = count_tokens(prompt_text)
prompt_output_tokens = count_tokens(prompt_text)

input_cost_r1 = 0.50
output_cost_r1 = 2.15

input_cost_r1 = 0.13
output_cost_r1 = 0.60


prompt_input_tokens_cost = input_cost_r1 * dollar_to_ruble / 10**6 # стоимость запроса к LLM в долларах за 1 млн токенов
prompt_output_tokens_cost = output_cost_r1 * dollar_to_ruble / 10**6 # стоимость запроса к LLM в долларах за 1 млн токенов


prompt_input_tokens_cost_rub = prompt_input_tokens * prompt_input_tokens_cost
prompt_output_tokens_cost_rub = prompt_output_tokens * prompt_output_tokens_cost

ocr_total_rub = ocr_ya * count_receipts
total_rub = ocr_total_rub + prompt_input_tokens_cost_rub + prompt_output_tokens_cost_rub

: 

In [72]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели DeepSeek R1:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 2 чеков модели DeepSeek R1:

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         0.02 ₽
  LLM обработка (output):        0.47 ₽
  --------------------------------------
  Итого:                         0.74 ₽


In [81]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 1 чеков модели Qwen3 MOE:

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽


In [74]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 2 чеков модели Qwen3 MOE:

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         0.02 ₽
  LLM обработка (output):        0.47 ₽
  --------------------------------------
  Итого:                         0.74 ₽


In [61]:
# Красивый вывод
print(f"Расчёт стоимости для {count_receipts} чеков модели Qwen3 MOE:\n")
print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
print(f"  LLM обработка (input):     {prompt_input_tokens_cost_rub:8.2f} ₽")
print(f"  LLM обработка (output):    {prompt_output_tokens_cost_rub:8.2f} ₽")
print("  --------------------------------------")
print(f"  Итого:                     {total_rub:8.2f} ₽")

Расчёт стоимости для 10 чеков модели Qwen3 MOE:

  OCR обработка чеков:           1.20 ₽
  LLM обработка (input):         0.11 ₽
  LLM обработка (output):        2.17 ₽
  --------------------------------------
  Итого:                         3.48 ₽


In [84]:
def calculate_receipt_processing_cost(
    text_receipt: str,
    count_receipts: int = 1,
    ocr_cost_per_receipt: float = 0.12,
    dollar_to_ruble: float = 80,
    input_cost_r1: float = 0.13,
    output_cost_r1: float = 0.60,
    name_model: str = "DeepSeek R1",
    count_tokens_func=None
):
    if count_tokens_func is None:
        raise ValueError("Необходимо передать функцию подсчёта токенов через аргумент `count_tokens_func`")

    prompt_text = f"""
# Role
Ты — ассистент по извлечению данных из текстов чеков о покупках. Тебе дан текст чека, который может содержать список покупок, их стоимость и другую информацию (даты, адреса, скидки, итоговые суммы и т.д.).

# Goal
Твоя задача — выделить из чека только следующие данные для каждой купленной позиции:
-позиция — наименование товара или услуги
=цена позиции — итоговая цена этого товара или услуги (без учета скидок, если указано отдельно)

#Instruction
1. Игнорируй итоги, суммы по скидкам, налоги, способы оплаты и прочую информацию, не являющуюся отдельной покупкой.
2. Если у позиции указано количество и цена за штуку, указывай итоговую цену (цена × количество).
3. Не включай товары с нулевой ценой или подарки.
4. Сохраняй структуру: одна строка — одна позиция.

# Формат ответа:
Для каждого чека составь список в формате:
Название позиции - цена итоговая

# Чеки
# Чек 1
{text_receipt * count_receipts}
"""

    # Подсчёт токенов
    input_tokens = count_tokens_func(prompt_text)
    output_tokens = input_tokens  # Примерно предполагается равный размер

    # Стоимость токенов (в рублях)
    input_cost_per_token_rub = input_cost_r1 * dollar_to_ruble / 1_000_000
    output_cost_per_token_rub = output_cost_r1 * dollar_to_ruble / 1_000_000

    input_cost_rub = input_tokens * input_cost_per_token_rub
    output_cost_rub = output_tokens * output_cost_per_token_rub

    ocr_total_rub = ocr_cost_per_receipt * count_receipts
    total_rub = ocr_total_rub + input_cost_rub + output_cost_rub

    # Красивый вывод
    print(f"Расчёт стоимости для {count_receipts} чеков модели {name_model}\n")
    print(f"  OCR обработка чеков:       {ocr_total_rub:8.2f} ₽")
    print(f"  LLM обработка (input):     {input_cost_rub:8.2f} ₽")
    print(f"  LLM обработка (output):    {output_cost_rub:8.2f} ₽")
    print("  --------------------------------------")
    print(f"  Итого:                     {total_rub:8.2f} ₽")

    # return {
    #     "ocr_rub": ocr_total_rub,
    #     "llm_input_rub": input_cost_rub,
    #     "llm_output_rub": output_cost_rub,
    #     "total_rub": total_rub,
    #     "input_tokens": input_tokens,
    #     "output_tokens": output_tokens
    # }


In [87]:
for k in range(1, 11):
    calculate_receipt_processing_cost(
        text_receipt=text_receipt,
        count_receipts=k,
        ocr_cost_per_receipt=ocr_ya,
        dollar_to_ruble=dollar_to_ruble,
        input_cost_r1=0.50,
        output_cost_r1=2.15,
        name_model="DeepSeek R1",
        count_tokens_func=count_tokens
    )


Расчёт стоимости для 1 чеков модели DeepSeek R1

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.05 ₽
  LLM обработка (output):        0.22 ₽
  --------------------------------------
  Итого:                         0.39 ₽
Расчёт стоимости для 2 чеков модели DeepSeek R1

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         0.09 ₽
  LLM обработка (output):        0.39 ₽
  --------------------------------------
  Итого:                         0.72 ₽
Расчёт стоимости для 3 чеков модели DeepSeek R1

  OCR обработка чеков:           0.36 ₽
  LLM обработка (input):         0.13 ₽
  LLM обработка (output):        0.57 ₽
  --------------------------------------
  Итого:                         1.06 ₽
Расчёт стоимости для 4 чеков модели DeepSeek R1

  OCR обработка чеков:           0.48 ₽
  LLM обработка (input):         0.17 ₽
  LLM обработка (output):        0.74 ₽
  --------------------------------------
  Итого:                         1.40 ₽


In [88]:
for k in range(1, 11):
    calculate_receipt_processing_cost(
        text_receipt=text_receipt,
        count_receipts=k,
        ocr_cost_per_receipt=ocr_ya,
        dollar_to_ruble=dollar_to_ruble,
        input_cost_r1=0.13,
        output_cost_r1=0.60,
        name_model="Qwen 3 MOE",
        count_tokens_func=count_tokens
    )


Расчёт стоимости для 1 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.12 ₽
  LLM обработка (input):         0.01 ₽
  LLM обработка (output):        0.06 ₽
  --------------------------------------
  Итого:                         0.19 ₽
Расчёт стоимости для 2 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.24 ₽
  LLM обработка (input):         0.02 ₽
  LLM обработка (output):        0.11 ₽
  --------------------------------------
  Итого:                         0.37 ₽
Расчёт стоимости для 3 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.36 ₽
  LLM обработка (input):         0.03 ₽
  LLM обработка (output):        0.16 ₽
  --------------------------------------
  Итого:                         0.55 ₽
Расчёт стоимости для 4 чеков модели Qwen 3 MOE

  OCR обработка чеков:           0.48 ₽
  LLM обработка (input):         0.04 ₽
  LLM обработка (output):        0.21 ₽
  --------------------------------------
  Итого:                         0.73 ₽
Расч